In [1]:
import pandas as pd
import pickle
import os
import numpy as np
from tqdm import tqdm
import spacy
import scispacy


In [255]:
nlp = spacy.load('en_core_sci_md')

In [223]:
data_dir = '/media/My1TBSSD1/MIMIC-CXR/'

In [472]:
df = pd.read_csv('../../../MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv.gz')

In [473]:
df = df.sort_values(by = 'study_id')


In [474]:
df[df.study_id >= 58850200]

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
10662,10476303,58850255,NaN,NaN,NaN,1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,NaN
47726,12095092,58850349,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
225453,19895187,58850368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
111934,14932641,58850456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
219161,19634294,58850558,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116115,15114531,59999832,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,1.0,0.0,NaN,0.0,NaN,1.0
39098,11738518,59999849,-1.0,1.0,NaN,-1.0,-1.0,NaN,NaN,1.0,NaN,1.0,NaN,-1.0,0.0,1.0
184240,18081075,59999880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
123449,15411545,59999888,NaN,0.0,NaN,0.0,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [42]:
df_split = pd.read_csv('../../../MIMIC-CXR/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv.gz')

In [54]:
test_ids = df_split[df_split.split=='test'].study_id.values

In [475]:
negation_dataset = df[df['No Finding']==0]

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices


In [120]:
negation_dataset[negation_dataset.study_id==57375967].iloc[0].to_dict()

{'subject_id': 10000764.0,
 'study_id': 57375967.0,
 'Atelectasis': nan,
 'Cardiomegaly': nan,
 'Consolidation': 1.0,
 'Edema': nan,
 'Enlarged Cardiomediastinum': nan,
 'Fracture': nan,
 'Lung Lesion': nan,
 'Lung Opacity': nan,
 'No Finding': nan,
 'Pleural Effusion': nan,
 'Pleural Other': nan,
 'Pneumonia': -1.0,
 'Pneumothorax': nan,
 'Support Devices': nan}

In [59]:
neg_test_dataset = negation_dataset[negation_dataset.study_id.isin(test_ids)]

In [119]:
neg_test_dataset[neg_test_dataset.study_id==57375967]

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices


In [2]:
with open(os.path.join('../../../MIMIC-CXR/lm_reports/class_label_mit_v3.pkl'), 'rb') as f:
    dataset = pickle.load(f)

In [482]:
dataset['neg_label'] = neg_labels

In [3]:
len(dataset['manual_neg_label_with_zeros'])

493

In [7]:
dataset['split']['manual_with_zeros']

493

In [477]:
class_to_idx = {
            'Atelectasis': 0, 'Cardiomegaly': 1, 'Consolidation': 2, 'Edema': 3, 'Enlarged Cardiomediastinum': 4, 
            'Fracture': 5, 'Lung Lesion': 6, 'Lung Opacity': 7, 'No Finding': 8, 'Pleural Effusion': 9, 
            'Pleural Other': 10, 'Pneumonia': 11, 'Pneumothorax': 12, 'Support Devices': 13
        }

classes = list(class_to_idx.keys())

In [478]:
count = 0
neg_labels = {}
for key in tqdm(dataset['label'].keys()):
    ds_row = df[df['study_id']==key]
    
    ds_dict = ds_row.iloc[0].to_dict()
    if 0 not in ds_dict.values():
        val_list = np.zeros(14)
    else:
        val_list = np.array([1 if ds_dict[cls] == 0 else 0 for cls in classes])
    count+=1
    neg_labels[key] = val_list

100%|██████████| 225286/225286 [03:33<00:00, 1053.69it/s]


In [480]:
# ManualNegationDataset
neg_labels

{50414267.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 53189527.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 53911762.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 56699142.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 57375967.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 50771383.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 54205396.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 50578979.0: array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 51178377.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 55697293.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 56164612.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 56522600.0: array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 58219844.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 50985099.0: a

In [437]:
manual_gt[manual_gt.SimpleTestReportID==arr[56140866]]

,SimpleTestReportID,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Airspace Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,TEXT
485,5933056,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Consolidation in the right base is most consis...


In [456]:
count = 0
manual_neg_labels = {}
for key in tqdm(arr.keys()):
    ds_row = manual_gt[manual_gt.SimpleTestReportID==arr[key]]
    
    if ds_row.shape[0] == 0:
        val_list = np.zeros(14)
    else:
        ds_dict = ds_row.iloc[0].to_dict()
        if 0 not in ds_dict.values():
            val_list = np.zeros(14)
        else:
            val_list = np.array([1 if ds_dict[cls] == 0 else 0 for cls in classes])
    count+=1
    manual_neg_labels[key] = val_list
#     if count == 5:
#         break

100%|██████████| 493/493 [00:00<00:00, 1598.90it/s]


In [458]:
len(manual_neg_labels)

493

In [170]:
arr = []
for key in tqdm(dataset['neg_label'].keys()):
    if dataset['neg_label'][key] is None:
        continue
    arr.append(dataset['neg_label'][key])

100%|██████████| 225286/225286 [00:00<00:00, 1899088.30it/s]


In [171]:
len(arr)

72303

In [181]:
np.sum(arr, axis = 0)

array([  751, 11169,  7665, 20839,  3523,   937,   644,  3375,     0,
       20825,    89, 14965, 37114,  2857])

In [173]:
Q_c = np.array([43590, 42520, 10207, 25470, 6760, 4189, 5931, 48778, 72528, 51348, 1894, 15677, 9927, 63100])

In [219]:
manual_gt = pd.read_csv('../../../MIMIC-CXR/lm_reports/mimic-groundtruth.csv')

In [244]:
manual_gt

,SimpleTestReportID,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Airspace Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,TEXT
0,5885021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"Slowly progressive chronic right pneumonia, co..."
1,5885040,NaN,1.0,1.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1. Mild left basal atelectasis; no pneumonia....
2,5885289,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No acute cardiopulmonary process.
3,5885836,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"In comparison with the study of 11/4, the card..."
4,5885875,NaN,NaN,1.0,NaN,1.0,-1.0,-1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,"Interval increase/development of bilateral, ri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,6208653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Essentially unchanged right paramediastinal fi...
683,6218645,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1. Right lower lobe fibrosis.\n 2. Moderate ...
684,6230895,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,The patient had left lower lobe lobectomy in _...
685,6234541,NaN,NaN,1.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,NaN,"As compared to the previous radiograph, there ..."


In [251]:
def remove_whitespace(line):
    return str(" ".join(line.split()).strip())

def list_to_string(sentence):
    return " ".join(sentence)

def normalize_report(row):
    report = row
    report_sentences = nlp(report)
    new_report_sentences = []
    for sentence in report_sentences.sents:
        index_to_keep_dict = {} # index: {keep that token or not, replace_with}
        for index in range(0, len(sentence)):
            token = sentence[index]
            if index < len(sentence) - 1:
                next_token = sentence[index + 1]
                if token.is_punct and next_token.is_punct and token.text.strip() == next_token.text.strip():
                    # when it is the same type of punctuation
                    index_to_keep_dict[index] = {'keep': False, 'replace_with': None}
                    continue
            if token.like_num:
                index_to_keep_dict[index] = {'keep': True, 'replace_with': 'NUMBER'}
            else:
                index_to_keep_dict[index] = {'keep': True, 'replace_with': None}
        # generate a new sentence based on this replacement
        new_sentence = []
        for index in range(0, len(sentence)):
            token = sentence[index]
            if not index_to_keep_dict[index]['keep']:
                continue # don't append when there is a double punctuation happening
            if index_to_keep_dict[index]['replace_with'] is not None:
                new_sentence.append(index_to_keep_dict[index]['replace_with'])
                continue
            new_sentence.append(token.text)
        s = list_to_string(new_sentence).strip()
        s = s.replace('DEID', '')
        s = remove_whitespace(s)
        new_report_sentences.append(s)
    return {'sentences': ' '.join(new_report_sentences)}

In [386]:

for key in tqdm(preprocessed_data['data_dict'].keys()):
    if 'AThe size of the pneumothorax at the right lung apex is smaller'.lower() in preprocessed_data['data_dict'][key]['text'].lower():
        print(key)
        break

100%|██████████| 225293/225293 [00:00<00:00, 734486.19it/s]


In [383]:
preprocessed_data['data_dict'][54805682]['text']

'Portable AP image of the chest . There has been interval placement of a right-sided chest tube which crosses the right lung base and impinges on the mediastinum . Haziness is seen in the right upper lung , which has an convex margin inferiorly and may represent loculated pleural effusion . No pneumothorax is seen . No dependent pleural fluid is seen . Opacities in the left lung base consistent with atelectasis . The cardiomediastinal silhouette is unremarkable . NUMBER . Interval placement of right-sided chest tubes which cross the right lung base impinges and mediastinum . NUMBER . Haziness in the right upper lung , which may represent loculated pleural effusion . A lateral radiograph could be obtained to further evaluate this opacity .'

In [381]:
normalize_report(manual_gt[manual_gt.SimpleTestReportID==5889236].TEXT.values[0].replace('_',''))['sentences']

'A right-sided chest tube is present with the distal end near the right lung apex . Right central line ends at lower SVC . Innumerable , bilateral , nodular opacities are similar . The size of the pneumothorax at the right lung apex is smaller whereas at the right lower lateral chest wall and at the right lung base is overall unchanged . Spinal hardware device is present at lower thoracic and upper lumbar region . Increase retrocardiac density representing left lower lung volume loss , moderate left and mild right pleural effusions are stable .'

In [303]:
normalize_report(record.TEXT.replace('1.','').strip())['sentences']

'Since _ , moderate-to-large right pleural effusion with right lung atelectasis and left lower lung volume loss reflected as increased retrocardiac density are unchanged . Left upper lung is clear . Mildly enlarged heart , mediastinal and hilar contours are unchanged .'

In [225]:
with open(os.path.join(data_dir,'lm_reports/mimic_dataset_mit_normalized.pkl'),'rb') as f2:
    preprocessed_data = pickle.load(f2)

In [376]:
manual_ids = []

for idx in tqdm(range(manual_gt.shape[0])):
    record = manual_gt.iloc[idx]
    manual_id = record.SimpleTestReportID
    manual_text = normalize_report(record.TEXT.replace('1.','').replace('_','').strip())['sentences']
#     print(manual_text)
    Found = False
    for key in preprocessed_data['data_dict'].keys():
        if manual_text in preprocessed_data['data_dict'][key]['text']:
#             print(key, preprocessed_data['data_dict'][key])
            manual_ids.append([key, manual_id, preprocessed_data['data_dict'][key]['text'], manual_text])
            Found = True
            break
            
    if not Found:        
        manual_text = manual_text.split('.')[0].strip()
        for key in preprocessed_data['data_dict'].keys():
            if manual_text in preprocessed_data['data_dict'][key]['text']:
    #             print(key, preprocessed_data['data_dict'][key])
                manual_ids.append([key, manual_id, preprocessed_data['data_dict'][key]['text'], manual_text])
                Found = True
                break
            
    if not Found:
        manual_ids.append([None, manual_id, None, manual_text])

100%|██████████| 687/687 [02:12<00:00,  5.18it/s]


In [377]:
manual_mapping = pd.DataFrame(manual_ids)

In [378]:
manual_mapping.columns = ['key','manual_id', 'preprocessed_data', 'manual_text']
manual_mapping

,key,manual_id,preprocessed_data,manual_text
0,58085167.0,5885021,A cluster of heterogeneous opacities in the ri...,"Slowly progressive chronic right pneumonia , c..."
1,57798090.0,5885040,Heterogeneous left basilar opacities do not ha...,Mild left basal atelectasis ; no pneumonia . N...
2,50414267.0,5885289,"There is no focal consolidation , pleural effu...",No acute cardiopulmonary process .
3,NaN,5885836,None,"In comparison with the study of NUMBER , the c..."
4,59039129.0,5885875,Frontal and lateral views of the chest are obt...,"Interval increase/development of bilateral , r..."
...,...,...,...,...
682,NaN,6208653,None,Essentially unchanged right paramediastinal fi...
683,59358922.0,6218645,NUMBER . Right lower lobe fibrosis . NUMBER . ...,Right lower lobe fibrosis . NUMBER . Moderate ...
684,53225437.0,6230895,The patient had left lower lobe lobectomy in ....,The patient had left lower lobe lobectomy in ....
685,55397704.0,6234541,"As compared to the previous radiograph , there...","As compared to the previous radiograph , there..."


In [379]:
manual_mapping[manual_mapping.key.isnull()]

,key,manual_id,preprocessed_data,manual_text
3,NaN,5885836,None,"In comparison with the study of NUMBER , the c..."
5,NaN,5886053,None,Early right middle lobe focal pneumonia
54,NaN,5889236,None,A right-sided chest tube is present with the d...
56,NaN,5889579,None,New Port-A-Cath terminating in the low SVC wit...
62,NaN,5890239,None,Comparison is made with a study of ; the area ...
...,...,...,...,...
638,NaN,6018037,None,Moderate right apical pneumothorax has very mi...
639,NaN,6018153,None,"As compared to the previous radiograph , the k..."
640,NaN,6019178,None,"As compared to the previous radiograph , there..."
641,NaN,6019671,None,"Left lung collapse , highly suspicious for an ..."


In [311]:
54244537,5885836
None, 5886053


454

In [395]:
with open(os.path.join(data_dir, 'lm_reports/class_label_mit_v2.pkl'), 'rb') as f:
    cls_dataset = pickle.load(f)

In [427]:
count = 1
arr = {}
for key in cls_dataset['split']['test']:
#     print('{0:10d}'.format(key),end='\r')
    if key in manual_mapping.key.dropna().values:
#         print(key, ': is used in val')
        print('{0:3d}'.format(count),end='\r')
        count+=1
        arr[key] = manual_mapping[manual_mapping.key == key].manual_id.values[0]

In [428]:
arr

{52835225: 5907463,
 53452091: 6128953,
 56140866: 5933056,
 52824884: 5976894,
 56446284: 5924365,
 57949791: 5969277,
 58117612: 5914386,
 58736291: 5901170,
 51129150: 6184479,
 55811525: 6238150,
 56498272: 6180990,
 56486000: 5901118,
 59775769: 5907866,
 56172325: 5909812,
 50563564: 5901075,
 51185902: 5898829,
 52363927: 5899787,
 53818026: 5899268,
 55878458: 5965924,
 59089311: 5899904,
 59170987: 5900379,
 51837636: 5927864,
 54594848: 5897804,
 58393560: 5966817,
 57163975: 6100247,
 52894975: 5944101,
 58072789: 5961626,
 52402828: 5933785,
 55438657: 5975433,
 59281953: 5908587,
 55911959: 5964712,
 56820999: 5962746,
 50078440: 5920810,
 50146341: 5922642,
 51656138: 5925180,
 56258422: 5927316,
 56603583: 5927257,
 57185571: 5927083,
 58274962: 5930608,
 53407845: 5907525,
 53537165: 5905639,
 53884408: 5907203,
 50533006: 5922225,
 52842984: 5928595,
 59345943: 5926965,
 59627448: 6003246,
 59983953: 5937137,
 50567642: 5886728,
 57001251: 5920936,
 55743226: 5984433,


In [392]:
601 - 

84 in train, 
492 - test

array([58085167., 57798090., 50414267., 59039129., 57075868., 58270659.,
       51526655., 58324748., 54587031., 59968351., 58701930., 55893591.,
       56630223., 50567642., 55027268., 52971492., 51328698., 50414267.,
       55023879., 55111273., 56732549., 50916783., 54164323., 50520166.,
       53100359., 50790949., 50348450., 55418359., 58005336., 54766893.,
       55906329., 52312858., 50664785., 53261045., 55153576., 53958673.,
       57673768., 52682048., 55469953., 57438544., 51322686., 52020944.,
       54831516., 56115450., 59763671., 56651744., 54831516., 56862577.,
       56093476., 51131705., 51006959., 55438661., 58125581., 59027235.,
       55011437., 50414267., 58490673., 59691119., 57765050., 58008930.,
       56264253., 53887723., 59089386., 51924292., 51351495., 57537037.,
       50712381., 54844091., 57053848., 52697084., 56599347., 55828202.,
       59712299., 50414267., 51766355., 57086338., 52991108., 52944435.,
       55009631., 59553780., 57142742., 51030092., 